In [1]:
import csv
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial import Delaunay
from igraph import *
import matplotlib.pyplot as plt
import json
import numpy as np
import os
import random
import math

### Q6

#### create graph

In [2]:
directory = os.path.abspath('/Users/bournsh/Desktop/UCLA_Studying/2020spring/ECE232/Project4/data/test_data')
df = pd.read_csv(directory + '/san_francisco-censustracts-2017-4-All-MonthlyAggregate.csv')

In [3]:
df = df.iloc[:, 0:4]
df = df.loc[df['month']==12]
dec_df = df.drop(['month'], axis=1)


In [4]:
dec_df.head()

,sourceid,dstid,mean_travel_time
0,1982,73,350.59
1,2213,1523,872.92
2,438,433,2262.35
3,448,333,1337.10
6,2231,315,805.52


In [5]:
init_dec_df = dec_df.copy()
init_dec_df['sourceid'] = init_dec_df['sourceid'].astype(str)
init_dec_df['dstid'] = init_dec_df['dstid'].astype(str)


In [6]:
tuples_init = [tuple(x) for x in init_dec_df.values]
g_init = Graph.TupleList(tuples_init, directed = False, edge_attrs = 'weight')

In [7]:
summary(g_init)

IGRAPH UNW- 1905 555265 -- 
+ attr: name (v), weight (e)


#### clear the duplicates

In [8]:
dec_array = np.asarray(dec_df) 
existing_paths = {}
existing_locations = np.unique(np.concatenate([dec_array[:,0],dec_array[:,1]]))
counter = 0
for row in dec_array:
    locations = tuple(np.sort(row[0:2]))
    if locations in existing_paths:
        existing_paths[locations].append(row[2])
        counter += 1
    else:
        existing_paths[locations] = [row[2]]

with open(directory + '/december_data.txt','w') as f:
    for locations in existing_paths:
        string = '{} {} {:.2f}\n'.format(int(locations[0]),int(locations[1]),np.mean(existing_paths[locations]))
        f.write(string)

In [9]:
g = Graph.Read(directory + '/december_data.txt', format='ncol', directed=False)
gcc = g.components().giant()
summary(g)
summary(gcc)

IGRAPH UNW- 1905 321713 -- 
+ attr: name (v), weight (e)
IGRAPH UNW- 1898 321703 -- 
+ attr: name (v), weight (e)


#### get the json file

In [10]:
with open(directory+'/san_francisco_censustracts.json', 'r') as f:
    data_json = json.load(f)
data_json.keys()

dict_keys(['type', 'features'])

In [11]:
geo_df = data_json['features']

In [12]:
colname = ['ID', 'Display_Name', 'Longitude', 'Latitude']
map_df = pd.DataFrame(columns = colname)
map_df

,ID,Display_Name,Longitude,Latitude


In [13]:
print(geo_df[0].keys())
print(geo_df[0]['properties'].keys())
print(geo_df[0]['geometry'].keys())

dict_keys(['type', 'properties', 'geometry'])
dict_keys(['MOVEMENT_ID', 'DISPLAY_NAME'])
dict_keys(['type', 'coordinates'])


In [14]:
i = 1
for item in geo_df:
    map_df.loc[i, 'ID'] = item['properties']['MOVEMENT_ID']
    map_df.loc[i, 'Display_Name'] = item['properties']['DISPLAY_NAME']
    coors = item['geometry']['coordinates'][0][0]
    if isinstance(coors[0], list):
        longitude = 0
        latitude = 0

        for coor in coors:
            longitude = longitude+coor[0]
            latitude = latitude+coor[1]
            
        longitude = longitude/len(coors)
        latitude = latitude/len(coors)       
    else:
        coors = item['geometry']['coordinates'][0]
        longitude = 0
        latitude = 0

        for coor in coors:
            longitude = longitude+coor[0]
            latitude = latitude+coor[1]
            
        longitude = longitude/len(coors)
        latitude = latitude/len(coors) 
        
    map_df.loc[i, 'Longitude'] = longitude
    map_df.loc[i, 'Latitude'] = latitude    
    i+=1

In [15]:
map_df

,ID,Display_Name,Longitude,Latitude
1,1,"Sargent Creek, San Ardo",-120.789,36.0716
2,2,"400 Northumberland Avenue, Redwood Oaks, Redwo...",-122.219,37.4696
3,3,"18300 Sutter Boulevard, Morgan Hill",-121.66,37.1505
4,4,"2700 Stoughton Way, Sheffield, Sacramento",-121.343,38.5731
5,5,"3200 Huntsman Drive, Rosemont Park, Sacramento",-121.362,38.5544
...,...,...,...,...
2706,2706,"10000 Bobbell Drive, Elk Grove",-121.467,38.3947
2707,2707,"400 Ericwood Court, Manteca",-121.222,37.82
2708,2708,"200 Westview Drive, Sunshine Gardens, South Sa...",-122.441,37.6627
2709,2709,"1200 Halyard Drive, West Sacramento",-121.563,38.5678


#### map geo_coordinates with map information

In [16]:
gvs = g.vs()
for v in gvs:
    v["Display_Name"] = map_df.loc[map_df["ID"] == v['name']].iloc[:,1].values[0]
    v["Coordinate"] = (map_df.loc[map_df["ID"] == v['name']].iloc[:,2].values[0],
                       map_df.loc[map_df["ID"] == v['name']].iloc[:,3].values[0])
gcc = g.components().giant()                    

In [17]:
summary(g)
summary(gcc)


IGRAPH UNW- 1905 321713 -- 
+ attr: Coordinate (v), Display_Name (v), name (v), weight (e)
IGRAPH UNW- 1898 321703 -- 
+ attr: Coordinate (v), Display_Name (v), name (v), weight (e)


### Q7

In [18]:
mst = gcc.spanning_tree(weights = gcc.es['weight'])
vertices = mst.vs()
summary(mst)



IGRAPH UNW- 1898 1897 -- 
+ attr: Coordinate (v), Display_Name (v), name (v), weight (e)


In [19]:
for i in range(6):
    r = random.randint(0, len(vertices))
    print("Random point{}:".format(i+1))
    print("One endpoint:\nID: {}, Display_Name: {}".format(vertices[r]["name"], vertices[r]["Display_Name"]))
#    print("Coordinate: {}".format(vertices[r]["Coordinate"]))
    print("Another endpoint:\nID: {}, Display_Name: {}".format(vertices[r+1]["name"], vertices[r+1]["Display_Name"]))
    print('='*60)
#    print("Coordinate: {}\n".format(vertices[r+1]["Coordinate"]))

Random point1:
One endpoint:
ID: 157, Display_Name: 100 Cheltenham Way, South San Jose, San Jose
Another endpoint:
ID: 1387, Display_Name: 400 Fieldcrest Drive, South San Jose, San Jose
Random point2:
One endpoint:
ID: 158, Display_Name: 1700 Klamath Drive, Salinas
Another endpoint:
ID: 2420, Display_Name: 5200 Cleek Court, Richmond
Random point3:
One endpoint:
ID: 1874, Display_Name: 4900 Minas Drive, South San Jose, San Jose
Another endpoint:
ID: 2153, Display_Name: 3300 Holly Drive, Alum Rock, San Jose
Random point4:
One endpoint:
ID: 2444, Display_Name: 700 William Moss Boulevard, Weston Ranch, Stockton
Another endpoint:
ID: 2010, Display_Name: Pepperwood Preserve Road, Santa Rosa
Random point5:
One endpoint:
ID: 1982, Display_Name: 100 Monte Verde Drive, Vacaville
Another endpoint:
ID: 1523, Display_Name: 1000 Neilson Street, Albany
Random point6:
One endpoint:
ID: 465, Display_Name: 20100 Marshall Street, Castro Valley
Another endpoint:
ID: 468, Display_Name: 700 Via Aires, San L

### Q8

In [50]:
def is_trangle(max_index, combined_set, *args):
    current_set = ()
    indices = []
    weights = []
    for index in range(3):
        indices.append(random.randint(0, max_index))
    current_set = tuple(sorted(indices))
    
    while current_set in combined_set:
        for index in range(3):
            indices.append(random.randint(0, max_index))
        current_set = tuple(sorted(indices))
    
    for e in current_set:
        weights.append(gcc_es[e]['weight'])
    if (weights[0] < weights[1] + weights[2] and weights[1] < weights[0] + weights[2] and weights[2] < weights[0] + weights[1]):
        return True, weights, current_set
    else:
        return False, weights, current_set

    

In [51]:
max_idx = len(gcc.vs()) - 1
gcc_vertices = gcc.vs()
gcc_es = EdgeSeq(gcc)
num_triangle = 1000
total_set = set()
num_satisfied = 0
num = 0

In [52]:
while num<num_triangle:
    for i in range(3):
        satisfy, weights, current_set = is_trangle(max_idx, total_set)
        if satisfy:
            break
   # total_set.add(current_set)
    #print('='*50,'triangle',num+1)
    
#     edges = []
#     start_nodes = []
#     end_nodes = []
#     for e in current_set:
#         edges.append(gcc_es[e])
        
#         source_vertex = gcc.vs[gcc_es[e].source]
#         target_vertex = gcc.vs[gcc_es[e].target]
        
#         start_nodes.append(source_vertex['Display_Name'])
#         end_nodes.append(target_vertex['Display_Name'])
#     for i in range(3):
#         print(start_nodes[i], '---', end_nodes[i])
    if satisfy:
        num_satisfied+=1
        #print('satisfied')
    #else:
        #print('Unsatisfied')
        
    num+=1
print(str(num_satisfied/num_triangle))

0.957
